## Simulating In-House and AWS Encryption
---


In [7]:
def in_house_encrypt(data, in_house_key):
    # A mock function to represent encrypting data with an in-house key
    encrypted_data = f"encrypted({data})-with-in-house-key({in_house_key})"
    return encrypted_data

def aws_encrypt(data, aws_key):
    # A mock function to represent AWS encrypting the already encrypted data with its own key
    double_encrypted_data = f"aws-encrypted({data})-with-aws-key({aws_key})"
    return double_encrypted_data

def aws_decrypt(double_encrypted_data, aws_key):
    # A mock function to represent AWS decrypting the data with its own key
    encrypted_data = double_encrypted_data.replace(f"-with-aws-key({aws_key})", "")
    encrypted_data = encrypted_data.replace("aws-encrypted(", "")
    encrypted_data = encrypted_data.replace(")", "")
    encrypted_data +=")"
    return encrypted_data

def in_house_decrypt(encrypted_data, in_house_key):
    # A mock function to represent decrypting data with an in-house key
    print(f"debug >> {encrypted_data}")
    data = encrypted_data.replace(f"-with-in-house-key({in_house_key})", "")
    data = data.replace("encrypted(", "")
    data = data.replace(")", "")
    return data

# Example data and keys
file_content = "Hello, World context "
in_house_key = "companyXYZkey"
aws_key = "awsS3key"

# Step 1: In-house encryption
encrypted_with_in_house = in_house_encrypt(file_content, in_house_key)
print(f"1. Encrypted with in-house key: {encrypted_with_in_house}")

# Step 2: Sending to AWS and AWS encrypts it with its key
encrypted_with_aws = aws_encrypt(encrypted_with_in_house, aws_key)
print(f"2. Encrypted with AWS key: {encrypted_with_aws}")

# Step 3: AWS decrypts its layer of encryption when requested
aws_decrypted_data = aws_decrypt(encrypted_with_aws, aws_key)
print(f"3. Decrypted by AWS (AWS layer): {aws_decrypted_data}")

# Step 4: In-house decryption after downloading from AWS
final_data = in_house_decrypt(aws_decrypted_data, in_house_key)
print(f"4. Decrypted with in-house key (Final Data): {final_data}")

1. Encrypted with in-house key: encrypted(Hello, World context )-with-in-house-key(companyXYZkey)
2. Encrypted with AWS key: aws-encrypted(encrypted(Hello, World context )-with-in-house-key(companyXYZkey))-with-aws-key(awsS3key)
3. Decrypted by AWS (AWS layer): encrypted(Hello, World context -with-in-house-key(companyXYZkey)
debug >> encrypted(Hello, World context -with-in-house-key(companyXYZkey)
4. Decrypted with in-house key (Final Data): Hello, World context 


## Step 2: Encrypt the Model File Locally
Python code to encrypt the trained_model.h5 file using AWS KMS-generated data keys:


In [8]:
import boto3
from cryptography.fernet import Fernet

# Initialize a boto3 KMS client
kms_client = boto3.client('kms')

# Request a new data key for encryption
data_key = kms_client.generate_data_key(KeyId='alias/YourKmsKeyAlias', KeySpec='AES_256')

# Extract the plain data key for encryption
plain_text_data_key = data_key['Plaintext']
encrypted_data_key = data_key['CiphertextBlob']

# Use the plaintext data key with Fernet to encrypt the file
fernet = Fernet(plain_text_data_key)

# Load your model file
with open('trained_model.h5', 'rb') as model_file:
    model_data = model_file.read()

# Encrypt the model data
encrypted_model_data = fernet.encrypt(model_data)

# Save the encrypted model to a new file
with open('trained_model_encrypted.h5', 'wb') as encrypted_file:
    encrypted_file.write(encrypted_model_data)

# Store the encrypted data key securely
with open('data_key_encrypted.bin', 'wb') as key_file:
    key_file.write(encrypted_data_key)


ModuleNotFoundError: No module named 'boto3'

In [ ]:
## Step 3 , upload to S3
aws s3 cp trained_model_encrypted.h5 s3://your-bucket-name/data_model/
aws s3 cp data_key_encrypted.bin s3://your-bucket-name/data_model/


## Step 5 decrypt in Ec2

In [ ]:

from cryptography.fernet import Fernet
import boto3

# Initialize a boto3 KMS client
kms_client = boto3.client('kms')

# Initialize a boto3 client for S3
s3_client = boto3.client('s3')

# Download the model and encrypted data key from S3
s3_client.download_file('your-bucket-name', 'data_model/data_key_encrypted.bin', 'data_key_encrypted.bin')

s3_client.download_file('your-bucket-name', 'data_model/data_key_encrypted.bin', 'trained_model_encrypted.h5')


# open bin
with open('data_key_encrypted.bin', 'rb') as key_file:
    encrypted_data_key = key_file.read()

# Decrypt the data key
data_key_plaintext = kms_client.decrypt(CiphertextBlob=encrypted_data_key)['Plaintext']

# Use the plaintext data key with Fernet to decrypt the file
fernet = Fernet(data_key_plaintext)

# Load your encrypted model file
with open('trained_model_encrypted.h5', 'rb') as encrypted_file:
    encrypted_model_data = encrypted_file.read()

# Decrypt the model data
model_data = fernet.decrypt(encrypted_model_data)

# Save the decrypted model to a new file or load directly into memory
with open('trained_model.h5', 'wb') as model_file:
    model_file.write(model_data)